In [ ]:

# Dependency setup
import os
import pandas as pd
import numpy as np
import datetime as dt
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Date, ForeignKey

In [ ]:
# Removes any existing instance of the database that this notebook creates
db_instance="hawaii.sqlite"

## if file exists, delete it ##
if os.path.isfile(db_instance):
    os.remove(db_instance)
else:
    print("Hawaii database not created, continue to connection engine")

In [ ]:
# Defining the path for the pandas CSV imports
clean_meas_csv = "data_files/CLEAN_hawaii_measurements.csv"
clean_stat_csv = "data_files/CLEAN_hawaii_stations.csv"

In [ ]:
# Reading csv data into pandas dataframe
measurements_df = pd.read_csv(clean_meas_csv, header="infer",dtype={"station":"str","date":"str","prcp":"float","tobs":"float"})
measurements_df["date"] = pd.to_datetime(measurements_df["date"],format="%Y-%m-%d", errors="coerce")
stations_df = pd.read_csv(clean_stat_csv, header="infer", dtype={"station":"str","name":"str","latitude":"float","longitude":"float","elevation":"float"})

In [ ]:

# Creating class for the "stations" table
class Stations(Base):
    __tablename__="stations"
    station = Column(String(50), primary_key = True, unique = True, index= True)
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
# Creating dtype dictionary for Stations data
dict_stations = {"station":sa.String(50),
                 "name": sa.String(255),
                 "latitude":sa.Float(),
                 "longitude":sa.Float(),
                 "elevation":sa.Float(),
                }

In [ ]:

# Creating class for the "measurements" table
class Measurements(Base):
    __tablename__="measurements"
    id = Column(Integer, primary_key = True, unique = True, index = True)
    station = Column(String(50), ForeignKey("stations.station"))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)
    
# Creating dtype dictionary Measurements Data
dict_measurements = {"id":sa.Integer(),
                     "station":sa.String(50),
                     "date":sa.Date(),
                     "prcp":sa.Float(),
                     "tobs":sa.Float(),
                    }

In [ ]:

# Create a connection to a SQLite database
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# Creating the SQLITE data tables from the previously created engine
Base.metadata.create_all(engine)

In [ ]:
# Establishing pathway for dataframe to be inserted into SQLITE database
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
# Exporting pandas measurement dataframe to hawaii.sqlite database by iterating through Measurements class
for i in range(len(measurements_df)):
    session.add(
        Measurements(
            station=measurements_df.station[i], 
            date=measurements_df.date[i], 
            prcp=measurements_df.prcp[i], 
            tobs=measurements_df.tobs[i]))

In [ ]:
# Commiting records to the "measurements" table within the database
session.commit()

# Exporting pandas station dataframe to hawaii.sqlite database by iterating through Stations class
for i in range(len(stations_df)):
    session.add(
        Stations(station=stations_df.station[i], 
                 name=stations_df.name[i], 
                 latitude=stations_df.latitude[i], 
                 longitude=stations_df.longitude[i], 
                 elevation=stations_df.elevation[i]))